In [ ]:
import astropy.stats
import holoviews as hv
import hvplot.pandas
import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow
import pyarrow.parquet
import xarray as xr
import zarr

IDX = pd.IndexSlice

# Plotting

In [ ]:
%%time
labelwise_df = pyarrow.parquet.read_pandas(
    "/home/jqs1/scratch/jqs1/microscopy/220718/RBS_DEG_library_20x.nd2.threets.measurements/pos0.labelwise.parquet"
).to_pandas()

In [ ]:
labelwise_df.columns = ["/".join(col).strip() for col in labelwise_df.columns.values]

In [ ]:
d = labelwise_df.loc[
    IDX[:, :, :, 1:], :
]  # .groupby(["t", "trench_set", "trench"]).median()
# d = d[d["RFP-PENTA/mean"].between(1800, 2400)]
d = d[d["RFP-Penta/mean"].between(1000, 4000)]

In [ ]:
ds = hv.Dataset(d, ["t", "trench_set", "trench"])

In [ ]:
ds.to(hv.Curve, "t", "mask/axis_major_length").overlay(["trench_set", "trench"]).opts(
    width=800, height=400
)

In [ ]:
ds.to(hv.Curve, "t", "YFP-DUAL/mean").overlay(["trench_set", "trench"]).opts(
    width=800, height=400
)

In [ ]:
d.reset_index().hvplot("t", "YFP-DUAL/mean", by=["trench_set", "trench"])

In [ ]:
hv.Curve(d, kdims=["t"], vdims=["YFP-DUAL/mean"], groupby=["trench_set", "trench"])

# Heatmap

In [ ]:
d["RFP-PENTA/mean"].hvplot.hist(bins=100, bin_range=(0, 10000))

In [ ]:
d["YFP-DUAL/mean"].hvplot.hist(bins=100, bin_range=(0, 10000))

In [ ]:
filtered_cells = d.reset_index()

In [ ]:
mask.to_frame()

In [ ]:
mask

In [ ]:
d.reset_index(level=(2, 3)).loc[mask.to_frame()]

In [ ]:
mask

In [ ]:
d.loc[mask, :]

In [ ]:
d.groupby(["trench_set", "trench", "t"]).filter(
    lambda x: x.loc[IDX[:, :, 70]]["YFP-DUAL"].median() > 4000
)

In [ ]:
d[d.groupby(["trench_set", "trench", "t"]).filter(lambda x: x.loc[IDX[:,:,70]]["YFP-DUAL"].median() > 4000)#.loc[IDX[:,:,70]] > 4000]

In [ ]:
mask = (
    d.loc[IDX[:, :, 61:77]].groupby(["trench_set", "trench"])["YFP-DUAL/mean"].median()
    < 4000
)
dropped = d.reset_index(level=(2, 3)).where(mask).dropna()
dropped["t"] = dropped["t"].astype(np.int16)
dropped["label"] = dropped["label"].astype(np.int16)
dropped = dropped.set_index(["t", "label"], append=True)
filtered_cells = dropped

In [ ]:
filtered_cells = d

In [ ]:
%%time
channel = "YFP-DUAL/mean"
# channel = "RFP-PENTA/mean"
filtered_cells = d.reset_index()
# filtered_cells = d[d].reset_index()
bins = np.geomspace(filtered_cells[channel].min(), filtered_cells[channel].max(), 100)
heatmap = filtered_cells.groupby(["t"]).apply(
    lambda x: pd.Series(np.histogram(x[channel], bins=bins)[0], index=bins[:-1])
)
heatmap.columns.name = channel
# heatmap = xr.Dataset({row: heatmap.loc[row].T for row in heatmap.index.levels[0]})
heatmap = xr.DataArray(heatmap.T)

In [ ]:
heatmap.hvplot.quadmesh(cmap="blues", logy=True, logz=True, clim=(1, 1e4))

In [ ]:
heatmap.hvplot.quadmesh(cmap="blues", logy=True, logz=True, clim=(1, 1e4))

# Binned heatmaps

In [ ]:
induction_bins = np.geomspace(400, 15000, 8)

In [ ]:
induction_bins

In [ ]:
%%time
channel = "YFP-DUAL/mean"
# channel = "RFP-PENTA/mean"
bin_range = d[channel].min(), d[channel].max()
bins = np.geomspace(*bin_range, 100)
heatmaps = {}
for idx in range(len(induction_bins) - 1):
    induction_bin = tuple(induction_bins[idx : idx + 2])
    mask = (
        d.loc[IDX[:, :, 61:77]]
        .groupby(["trench_set", "trench"])["YFP-DUAL/mean"]
        .median()
        .between(*induction_bin)
    )
    dropped = d.reset_index(level=(2, 3)).where(mask).dropna()
    dropped["t"] = dropped["t"].astype(np.int16)
    dropped["label"] = dropped["label"].astype(np.int16)
    dropped = dropped.set_index(["t", "label"], append=True)
    filtered_cells = dropped  # .reset_index()
    heatmap = filtered_cells.groupby(["t"]).apply(
        lambda x: pd.Series(np.histogram(x[channel], bins=bins)[0], index=bins[:-1])
    )
    heatmap.columns.name = channel
    # heatmap = xr.Dataset({row: heatmap.loc[row].T for row in heatmap.index.levels[0]})
    # heatmaps[induction_bin] = xr.DataArray(heatmap.T)
    heatmaps[induction_bin] = xr.DataArray(heatmap.T)
heatmaps = xr.concat(
    heatmaps.values(),
    pd.Index([pd.Interval(*k) for k in heatmaps.keys()], name="induction_bin"),
)

In [ ]:
a = heatmaps.coords["induction_bin"]

In [ ]:
enumerate(a.values)

In [ ]:
list(enumerate(a.values))

In [ ]:
hv.HoloMap(
    {
        heatmaps.isel(induction_bin=idx).hvplot.quadmesh(cmap="blues", logy=True, logz=True, clim=(1, 1e4))
        for idx in range(len(heatmaps.coords["induction_bin"]))
    ]
)#.cols(1)

In [ ]:
hv.Layout(
    [
        heatmaps.isel(induction_bin=idx).hvplot.quadmesh(
            cmap="blues", logy=True, logz=True, clim=(1, 1e4)
        )
        for idx in range(len(heatmaps.coords["induction_bin"]))
    ]
).cols(1)

# Streamlines

In [ ]:
def get_limits(x):
    x = x.droplevel(0, axis=1)
    return pd.DataFrame(
        {
            "lower": x["median"] - x["median_absolute_deviation"],
            "upper": x["median"] + x["median_absolute_deviation"],
        }
    )

In [ ]:
filtered_cells = d
medians = filtered_cells.groupby(["t"]).agg(
    ["median", astropy.stats.median_absolute_deviation]
)

limits = medians.groupby(level=0, axis=1).apply(get_limits)

In [ ]:
observable = "YFP-DUAL/mean"

medians2 = medians[observable].reset_index()
limits2 = limits[observable].reset_index()
mean_plot = medians2.hvplot.line("t", "median", logy=True)
noise_plot = limits2.hvplot.area(
    x="t", y="lower", y2="upper", stacked=False, alpha=0.2, logy=True
)

In [ ]:
(mean_plot * noise_plot).opts(width=800, height=800)

In [ ]:
induction_bins = np.geomspace(400, 15000, 8)

In [ ]:
induction_bins

In [ ]:
%%time
channel = "YFP-DUAL/mean"
# channel = "RFP-PENTA/mean"
streamlines = {}
for idx in range(len(induction_bins) - 1):
    induction_bin = tuple(induction_bins[idx : idx + 2])
    mask = (
        d.loc[IDX[:, :, 61:77]]
        # d.loc[IDX[:, :, 42:51]]
        .groupby(["trench_set", "trench"])["YFP-DUAL/mean"]
        .median()
        .between(*induction_bin)
    )
    dropped = d.reset_index(level=(2, 3)).where(mask).dropna()
    dropped["t"] = dropped["t"].astype(np.int16)
    dropped["label"] = dropped["label"].astype(np.int16)
    dropped = dropped.set_index(["t", "label"], append=True)
    filtered_cells = dropped  # .reset_index()
    medians = filtered_cells.groupby(["t"]).agg(
        ["median", astropy.stats.median_absolute_deviation]
    )
    limits = medians.groupby(level=0, axis=1).apply(get_limits)
    medians2 = medians[channel].reset_index()
    limits2 = limits[channel].reset_index()
    mean_plot = medians2.hvplot.line("t", "median", logy=True)
    noise_plot = limits2.hvplot.area(
        x="t", y="lower", y2="upper", stacked=False, alpha=0.2, logy=True
    )
    streamlines[induction_bin] = mean_plot * noise_plot
streamlines = hv.HoloMap(streamlines, kdims=["min", "max"])

In [ ]:
streamlines.overlay().opts(width=900, height=400)

# Images

In [ ]:
img = zarr.open(
    zarr.LMDBStore(
        "/home/jqs1/scratch/jqs1/microscopy/220718/RBS_DEG_library_20x.nd2.threets.images/pos0.zarr"
    )
)

In [ ]:
plt.imshow(img["raw/1/60/RFP-Penta"][:, :, 50])

In [ ]:
plt.imshow(img["segmentation/1/60/RFP-Penta"][:, :, 50])

# FISH

In [ ]:
import h5py

In [ ]:
# ch, vmax = "RFP", 2000
ch, vmax = "Cy5", 6000
# ch, vmax = "Cy7", 3000
v = 30
t = 4
f = h5py.File(
    f"/home/jqs1/scratch/jqs1/microscopy/220704/FISH/real_run/fov={v}_config={ch}_t={t}",
    "r",
)

In [ ]:
img = np.asarray(f["data"])

In [ ]:
plt.figure(figsize=(20, 15))
plt.imshow(img, cmap="RdPu", vmax=vmax)